In [1]:
%matplotlib inline

# Mueller matrix

Exemplary fit of the complete Mueller matrix of a SiO2 on Si measurement.

In [2]:
import elli
from elli.fitting import ParamsHist, fit_mueller_matrix

# sphinx_gallery_thumbnail_path = '_static/mueller_matrix.png'

## Read data

We load the data from an ascii file containing each of the mueller matrix elements.
The wavelength range is cut to be in between 210 nm and 820 nm,
to stay in the range of the provided literature values for Si.
The data is expected to be in a pandas dataframe containing the columns Mxy,
where x and y refer to the matrix element inside the mueller matrix.
The data is scaled by the M11 element, such that $M_{11} = 1$ for all wavelengths.
To show the structure we print the `MM` dataframe.
If you load your data from another source make sure it adheres to this form.



In [3]:
MM = elli.read_spectraray_mmatrix("Wafer_MM_70.txt").loc[210:820]
print(MM)

            M11      M12      M13      M14      M21      M22      M23  \
Wavelength                                                              
210.30366   1.0 -0.09147 -0.01241 -0.00614 -0.09605  0.99426 -0.00364   
210.76102   1.0 -0.10879 -0.01151 -0.00901 -0.09754  0.99475 -0.00427   
211.21834   1.0 -0.10639 -0.00694 -0.00475 -0.10761  0.99637 -0.00207   
211.67561   1.0 -0.11544 -0.02094 -0.00348 -0.11467  0.99666 -0.00528   
212.13284   1.0 -0.12199  0.02182  0.00480 -0.12010  0.99680 -0.00437   
...         ...      ...      ...      ...      ...      ...      ...   
818.08934   1.0 -0.44285 -0.00048 -0.00288 -0.44048  0.99630 -0.00189   
818.50589   1.0 -0.44301  0.00376  0.00118 -0.44040  0.99688 -0.00274   
818.92240   1.0 -0.44245  0.00887  0.00626 -0.44024  0.99723 -0.00457   
819.33886   1.0 -0.44406 -0.00646 -0.00610 -0.44126  0.99661  0.00234   
819.75528   1.0 -0.44381  0.01170  0.00550 -0.44135  0.99606 -0.00653   

                M24      M31      M32      M33    

## Setting start parameters
Here we set the start parameters for the SiO2 cauchy dispersion
and thickness of the layer.



In [4]:
params = ParamsHist()
params.add("SiO2_n0", value=1.452, min=-100, max=100, vary=True)
params.add("SiO2_n1", value=36.0, min=-40000, max=40000, vary=True)
params.add("SiO2_n2", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_k0", value=0, min=-100, max=100, vary=True)
params.add("SiO2_k1", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_k2", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_d", value=120, min=0, max=40000, vary=True)

## Load silicon dispersion from the refractiveindexinfo database
You can load any material from the index
[refractiveindex.info](https://refractiveindex.info)_, which is
embedded into the software (so you may use it offline, too). Here, we
are interested in the literature values for the silicon substrate.
First we need to load the database with ``rii_db = elli.db.RII()`` and
then we can query it with ``rii_db.get_mat("Si", "Aspnes")`` to load
this
[entry](https://refractiveindex.info/?shelf=main&book=Si&page=Aspnes)_.



In [5]:
rii_db = elli.db.RII()
Si = rii_db.get_mat("Si", "Aspnes")

## Building the model
Here the model is build and the experimental structure is returned.
For details on this process please refer to the `Basic usage` example.
When executed in an jupyter notebook this displays an interactive graph
with which you can select the start parameters before fitting the data.



In [6]:
@fit_mueller_matrix(MM, params, display_single=False, sharex=True, full_scale=False)
def model(lbda, params):
    SiO2 = elli.Cauchy(
        params["SiO2_n0"],
        params["SiO2_n1"],
        params["SiO2_n2"],
        params["SiO2_k0"],
        params["SiO2_k1"],
        params["SiO2_k2"],
    ).get_mat()

    Layer = [elli.Layer(SiO2, params["SiO2_d"])]

    return elli.Structure(elli.AIR, Layer, Si).evaluate(
        lbda, 70, solver=elli.Solver4x4, propagator=elli.PropagatorExpm()
    )

## Plot & Fit the model
Here we plot the model at the initial parameter set vs. the experimental data.



In [7]:
model.plot()

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
              'type': 'scatter',
              'uid': '26152dd8-d954-4239-bd02-e6c7c23ea803',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([1., 1., 1., ..., 1., 1., 1.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#636EFA', 'dash': 'dash'},
              'name': 'M11 theory',
              'type': 'scatter',
              'uid': '04e82865-c248-4050-b290-24b784d4ad85',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([1., 1., 1., ..., 1., 1., 1.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#EF553B', 'dash': 'solid'},
             

We can also plot the residual between measurement and model.



In [8]:
model.plot_residual()

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
              'type': 'scatter',
              'uid': 'fcb845dd-761b-4cc0-9265-43d204f7570e',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#EF553B', 'dash': 'solid'},
              'name': 'M12 ',
              'type': 'scatter',
              'uid': '5b63291e-83f0-4cb3-bc2f-5e8739d6d663',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x2',
              'y': array([0.58002385, 0.59119998, 0.58045745, ..., 0.16502466, 0.16587253,
                          0.16486215], shape=(1396,)),
              'yaxis': 'y2'},
   

Now we execute a fit and plot the model afterwards.



In [9]:
fit_stats = model.fit()
model.plot(full_scale=False)

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
              'type': 'scatter',
              'uid': '2589ae26-cd44-46b5-8f53-136f6aa41a10',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([1., 1., 1., ..., 1., 1., 1.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#636EFA', 'dash': 'dash'},
              'name': 'M11 theory',
              'type': 'scatter',
              'uid': 'ab082c74-d3b4-4e48-acec-8279c4aef41d',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([1., 1., 1., ..., 1., 1., 1.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#EF553B', 'dash': 'solid'},
             

For comparison we plot the residual again to have a figure of merit
for the fit quality



In [10]:
model.plot_residual()

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
              'type': 'scatter',
              'uid': 'e1be276c-3b2a-41b6-afa4-49b62b3356e4',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.], shape=(1396,)),
              'yaxis': 'y'},
             {'line': {'color': '#EF553B', 'dash': 'solid'},
              'name': 'M12 ',
              'type': 'scatter',
              'uid': 'c996633b-553f-4dab-8b96-234228d07537',
              'x': array([210.30366, 210.76102, 211.21834, ..., 818.9224 , 819.33886, 819.75528],
                         shape=(1396,)),
              'xaxis': 'x2',
              'y': array([ 0.00065224,  0.01219155,  0.00420564, ..., -0.00481608, -0.00370779,
                          -0.00445863], shape=(1396,)),
              'yaxis': 'y2'

We may also print the fit statistics.



In [11]:
fit_stats

## References
[Here](https://github.com/PyEllips/pyElli/tree/master/examples/SiO2_Si%20Mueller%20Matrix)
you can find the latest jupyter notebook and data files of this example.

